# Fundamental Valuation using Python

## A Bottom-Up Approach with Financial Data and Multiples
This project provides a fundamental valuation of NVIDIA Corporation (NVDA) using Python tools and financial data sourced from the Yahoo Finance API via the yfinance library.

The analysis is designed to help investors and data analysts better understand the intrinsic value of a company by breaking down its financials, calculating key valuation metrics, and comparing market-implied values to accounting-based indicators.

## Carga inicial

In [ ]:
# Importación de librerías
import yfinance as yf
import pandas as pd

In [ ]:
# Crear objeto de NVIDIA
nvda = yf.Ticker("NVDA")

In [2]:
# Revisión de contenido (valores en los encabezados)
#headers = nvda.financials.index.tolist()
#for h in headers:
#  print(h)

## Extracción de datos

In [ ]:
# Función auxiliar para extraer un valor si existe
def safe_extract(df, row_name, column_name):
    if row_name in df.index:
        return df.loc[row_name, column_name]
    else:
        return None

# Extracción de datos - Cuenta de resultados
df = nvda.financials
column_name = df.columns[1]

total_revenue = safe_extract(df, "Total Revenue", column_name)
gross_profit = safe_extract(df, "Gross Profit", column_name)
ebitda = safe_extract(df, "EBITDA", column_name)
ebit = safe_extract(df, "EBIT", column_name)
net_income = safe_extract(df, "Net Income", column_name)

# Extracción de datos - Balance
df= nvda.balance_sheet
column_name = df.columns[0]

total_assets = safe_extract(df, "Total Assets", column_name)
total_lianilities = safe_extract(df, "Total Liabilities Net Minority Interest", column_name)
total_equity = safe_extract(df, "Total Equity Gross Minority Interest", column_name)
cash = safe_extract(df, "Cash And Cash Equivalents", column_name)

In [ ]:
# Revisión de datos
print("Resultados:")
print(f"Total Revenue: {total_revenue:,.0f}$")
print(f"Total Profit: {gross_profit:,.0f}")
print(f"EBITDA: {ebitda:,.0f}$")
print(f"EBIT: {ebit:,.0f}$")
print(f"Net Income: {net_income:,.0f}$")

print("\nBalance:")
print(f"Total Assets: {total_assets:,.0f}$")
print(f"Total Liabilities: {total_lianilities:,.0f}$")
print(f"Total Equity: {total_equity:,.0f}$")
print(f"Cash: {cash:,.0f}$")

Resultados:
Total Revenue: 60,922,000,000$
Total Profit: 44,301,000,000
EBITDA: 35,583,000,000$
EBIT: 34,075,000,000$
Net Income: 29,760,000,000$

Balance:
Total Assets: 111,601,000,000$
Total Liabilities: 32,274,000,000$
Total Equity: 79,327,000,000$
Cash: 8,589,000,000$


## Ratios contables

In [ ]:
roe = net_income / total_equity
print(f"ROE: {roe:.2%}")

cash_ratio = cash / total_assets
print(f"Cash Ratio: {cash_ratio:.2%}")

debt_ratio = total_lianilities / total_assets
print(f"Debt Ratio: {debt_ratio:.2%}")

profit_revenue = gross_profit / total_revenue
print(f"Profit (as % of Revenue): {profit_revenue:.2%}")

ebitda_revenue = ebitda / total_revenue
print(f"EBITDA (as % of Revenue): {ebitda_revenue:.2%}")

ROE: 37.52%
Cash Ratio: 7.70%
Debt Ratio: 28.92%
Profit (as % of Revenue): 72.72%
EBITDA (as % of Revenue): 58.41%
